<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/ResNeXt101_32x8d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
'''IMPORTING NECESSARY MODULES
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('./Trainers/')
sys.path.append('./Dataloaders/')
sys.path.append('./utils/')
sys.path.append('./architectures/')

from dataloader import mydataset, create_prime_dict 
from trainer_ResNet import train, test_classify
from resnet_models import ResNet,Bottleneck,resnext101_32x8d
from Load_model import load
from plot_curves import plot_loss, plot_acc

In [ ]:
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:6


**Dataloading Scheme**

In [ ]:
trainlist = 'add path of training list'
validlist = 'add path of validation list'

In [ ]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  2


In [ ]:
'''Train Dataloader''' 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 128, num_workers=16,pin_memory=True)


'''Validation Dataloader''' 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

**Model Definition**

Train Resnet50/ ResNext101

In [ ]:
modelname = 'ResneXt101_32x8d'
modelpath = './saved_model_checkpoints/'+modelname

In [ ]:
# model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)


'''
Pretrained ResNeXt101_32x8d
'''
model = resnext101_32x8d(pretrained = True)


model = nn.DataParallel(model,device_ids=[6,7]).to(device)
model


# '''
# Load saved model from checkpoint
# '''
# model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)

In [ ]:
'''
Transfer Learning
'''

# for param in model.module.parameters():
#     param.requires_grad = False

fc_inputs = model.module.fc.in_features

model.module.fc = nn.Sequential(
    nn.Linear(fc_inputs, 2)
#     nn.BatchNorm1d(4096),
#     nn.ReLU(),
#     nn.Linear(768, 2)
    
)

model.to(device)

**Hyperparameters**

In [ ]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 20, gamma = 0.1)

# Epochs
num_Epochs = 80

In [ ]:
writer = SummaryWriter("ResNeXt101_32x8d")

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)\

writer.flush()
writer.close()

In [ ]:
'''
Load saved model from checkpoint
'''
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)

**Evaluate**

In [ ]:
test_classify(model, validation_dataloader, criterion, device)

(0.7325438114110836, 0.5090180360721442)

In [ ]:
avg_loss = 0

for batch_num, (feats, captions, input_id, attention_masks, target) in enumerate(train_dataloader):
        feats, target = feats.to(device), target.to(device)


        '''
        Compute output and Loss
        '''
        output, embeddings = model(feats) 
        total_loss = criterion(output, target)
        
        
        avg_loss += total_loss.item()
        
        
avg_loss/len(train_dataloader)

0.6497447961255124

In [ ]:
'''
Scale of embeddings
'''
embeddings[0][0:100]

tensor([-1.2928e-02, -2.5917e-04, -7.6083e-03, -2.3284e-02, -1.7803e-02,
        -1.1721e-02, -1.4128e-02, -3.9467e-03,  6.5848e-03, -4.5360e-03,
        -5.9023e-03, -2.7679e-02, -1.8074e-02,  2.7270e-03, -1.8123e-02,
        -2.0154e-02,  1.9357e-03, -1.4437e-02, -2.0706e-02, -4.4435e-02,
         2.3793e-02,  1.9012e-03, -1.5128e-02, -2.1997e-02, -1.7046e-02,
         1.6173e-03, -1.6588e-02,  3.5790e-02,  2.8867e-03, -2.5260e-02,
        -1.3732e-02, -8.6504e-03, -1.4730e-02, -1.7183e-02, -1.4189e-02,
        -2.1576e-03, -1.1566e-02, -2.0425e-05, -9.7702e-03, -1.0505e-02,
        -2.9042e-02, -3.8561e-03, -5.6513e-03, -1.2800e-02,  4.6991e-04,
        -2.6957e-03,  2.3134e-03, -2.1837e-02, -1.2266e-02,  6.1447e-04,
        -3.7410e-03, -1.0256e-02, -2.3636e-03, -4.2802e-02,  3.3450e-02,
        -1.0314e-02, -4.2708e-03,  1.0251e-03,  1.9456e-02, -2.2669e-02,
        -1.3707e-02,  5.0964e-03,  1.7879e-03, -2.2759e-02,  2.6169e-03,
        -4.8373e-02, -2.1960e-03, -2.8194e-02, -1.3

In [ ]:
'''
Scale of output
'''
output[0]

tensor([ 0.2626, -0.2895], device='cuda:6', grad_fn=<SelectBackward>)